In [ ]:
import numpy as np
from numpy import sqrt, mean, absolute
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Dataset



##training

In [ ]:
df = pd.read_excel('beras_train.xlsx', engine='openpyxl')
df

##testing

In [ ]:
dft = pd.read_excel('beras_test.xlsx', engine='openpyxl')
dft

# Normalisasi


##Training

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# default range 0-1
mmscaler = MinMaxScaler()
dataset_norm = mmscaler.fit_transform(df)
pd.DataFrame(dataset_norm)

##Testing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# default range 0-1
mmscaler = MinMaxScaler()
dataset_tes_norm = mmscaler.fit_transform(dft)
pd.DataFrame(dataset_tes_norm)

# Pembagian data input dan target


##training

In [ ]:
X = dataset_norm[:,[0,1,2,3,4,5,6,7,8,9,10,11]]
y_norm = dataset_norm[:, [-1]]
y = y_norm.reshape(24)

##testing

In [ ]:
x_tes = dataset_tes_norm[:,[0,1,2,3,4,5,6,7,8,9,10,11]]

# K-Fold Cross validation

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
for train_index, test_index in cv.split(X,y):
 print("TRAIN:", train_index, "TEST:", test_index)
 X_train, X_test = X[train_index], X[test_index]
 y_train, y_test = y[train_index], y[test_index]

##Menampilkan hasil split data

In [ ]:
for train_index, test_index in cv.split(X,y):
   print("------------------------------------------------------------------------")
   print("Index TRAIN:", train_index, "Index TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]
   print("X_TRAIN: \n",X_train,"\n X_Test: \n",X_test,"\n Y_Train: \n", y_train,"\nY_Test: ", y_test)
   print("------------------------------------------------------------------------\n\n")

## Pembangunan Model Regresi

In [ ]:
## membangun model
model = KNeighborsRegressor(metric='euclidean')
model

# Evaluasi

In [ ]:
k_range = range(1, 15)
rmse_err = []
for k in k_range:
    model = KNeighborsRegressor(n_neighbors=k, metric='euclidean')    
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv)
    RMSE = sqrt(mean(absolute(scores)))
    rmse_err.append(RMSE) 
    print('RMSE value for k= ' , k , 'is:', RMSE)
plt.plot(k_range, rmse_err, '-ok')
plt.xlabel('Value of K for KNN')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()


#Performance Measure

In [ ]:
#akurasi RMSE
model = KNeighborsRegressor(n_neighbors=2, metric='euclidean')
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv)    
sqrt(mean(absolute(scores)))

# Model terbaik

In [ ]:
model = KNeighborsRegressor(n_neighbors=2, metric='euclidean')
model.fit(X, y)

# Prediksi Penjualan

In [ ]:
y_pred = model.predict(x_tes)
y_pred
dft['prediksi'] = y_pred
dft

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1.9,1.9])
Prediksi = np.array(dft['prediksi'])
Bulan = ['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 'Agustus', 'September', 'Oktober', 'November', 'Desember']

ax.bar(Bulan,Prediksi)
plt.show()